<a href="https://colab.research.google.com/github/suarim/Node_tut/blob/main/Hardcode_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/enhanced_dns4-16k.wav'

In [ ]:
num_speakers = 2
language = 'English'
model_size = 'large'
model_name = model_size

In [ ]:
!pip install librosa

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install speechbrain==0.5.16
!pip install pydub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: speechbrain
    Found existing installation: speechbrain 1.0.0
    Uninstalling speechbrain-1.0.0:
      Successfully uninstalled speechbrain-1.0.0


In [ ]:
import whisper
import datetime
import subprocess
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
import numpy as np

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
from scipy.spatial.distance import pdist, squareform
from datetime import timedelta

In [ ]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [ ]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.88G/2.88G [00:42<00:00, 72.9MiB/s]


In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [ ]:
audio = Audio()
def segment_embedding(segment):
  start = segment["start"]
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)

  return embedding_model(waveform[None])

In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)
embeddings = np.nan_to_num(embeddings)

In [ ]:
distance_matrix = squareform(pdist(embeddings, metric='euclidean'))
def agglomerative_clustering(distance_matrix, num_clusters):
    num_points = distance_matrix.shape[0]
    clusters = [[i] for i in range(num_points)]
    while len(clusters) > num_clusters:
        min_distance = np.inf
        merge_candidates = (0, 0)
        for i in range(len(clusters)):
            for j in range(i + 1, len(clusters)):
                dist = np.mean([distance_matrix[p1, p2] for p1 in clusters[i] for p2 in clusters[j]])
                if dist < min_distance:
                    min_distance = dist
                    merge_candidates = (i, j)
        clusters[merge_candidates[0]].extend(clusters[merge_candidates[1]])
        del clusters[merge_candidates[1]]
    labels = np.zeros(num_points, dtype=int)
    for cluster_idx, cluster in enumerate(clusters):
        for point in cluster:
            labels[point] = cluster_idx
    return labels

num_speakers = 2
labels = agglomerative_clustering(distance_matrix, num_speakers)

for i in range(len(segments)):
    segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    print("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  print(segment["text"][1:] + ' ')



SPEAKER 1 0:00:00

What's your interest in it, Mr. Wayne? 

SPEAKER 2 0:00:03

I want to borrow it. For, uh, Spelunky. 

SPEAKER 1 0:00:07

Spelunky? 

SPEAKER 2 0:00:09

Yeah, you know, cave diving. 

SPEAKER 1 0:00:11

Expecting to run into much gunfire in these caves? 
